In [26]:
import (
	"context"
	"fmt"
    "log"
    "time"
	"go.mongodb.org/mongo-driver/bson"
	"go.mongodb.org/mongo-driver/mongo"
	"go.mongodb.org/mongo-driver/mongo/options"
    

)

In [9]:
type Realest struct {
	Title, Address, URL, DateTime string
	ID, Price                     int
	Info                          map[string]string
	Active                        bool
	Updates                       map[string]Realest // datetime string
}

type RealestKey struct {
	Title, Address string
	ID             int
}

func logIfErr(e error, msg string) bool {
	if e != nil {
		log.Fatal(msg+"\n", e)
	}
	return e != nil
}

func logIfFatal(e error, msg string) bool {
	if e != nil {
		log.Fatalf("%s %s", e)
	}
	return e != nil
}

In [10]:
func isInDb(c *mongo.Collection, realest Realest) (*mongo.SingleResult, bool) {
	// If realest in db, return the decoded realest and true, else nil and false.

	one := c.FindOne(context.TODO(), bson.M{
		"id":      realest.ID,
		"title":   realest.Title,
		"address": realest.Address,
	})
	return one, one.Err() == nil

}

func getFinnRealestateCollection() *mongo.Collection {
	clientOptions := options.Client().ApplyURI("mongodb://localhost:27017")
	client, e := mongo.Connect(context.TODO(), clientOptions)
	logIfErr(e, "client, e := mongo.Connect(context.TODO(), clientOptions), Failed.")

	e = client.Ping(context.TODO(), nil)
	logIfErr(e, "e = client.Ping(context.TODO(), nil), Failed")
	fmt.Println("Connected to mongoDB!")
	collection := client.Database("Finn").Collection("Realestate")
	fmt.Println("Collection:", collection)
	return collection
}

c := getFinnRealestateCollection()

Connected to mongoDB!
Collection: &{0xc001090380 0xc000d609c0 Realestate 0xc0005f3660 <nil> 0x7f774ff718e0 0xc000851340 0xc000851420 0xc00023d2d0}


In [27]:
func UpdateManyRealestate(c *mongo.Collection, realestates map[int]Realest) {
	var old Realest
	var isUpdated bool
	for _, realest := range realestates {
		if oldBson, exist := isInDb(c, realest); exist {
			// if realest exists in db, update if different
			oldBson.Decode(&old)
			if old.Updates[time.Now().String()], isUpdated = realest.RightUpdates(old); isUpdated  {
				// Insert update
				c.UpdateOne(context.TODO(), oldBson, old)

			} // Else skip

		} else {
			// Insert
			c.InsertOne(context.TODO(), realest)
		}
	}
}

ERROR: repl.go:8:53: not a package: "realest" in realest.RightUpdates <*ast.SelectorExpr>

In [16]:
old, isIn := isInDb(c, Realest{
    ID: 205519251,
    Title: "Oppussingsobjekt - Pen og koselig liten enebolig med garasje og usjenert bakhage",
    Address: "Geddes gate 9, 3189 Horten",
})

In [23]:
newRealest := Realest{
    ID: 10
}


{Oppussingsobjekt - Pen og koselig liten enebolig med garasje og usjenert bakhage Geddes gate 9, 3189 Horten https://www.finn.no/realestate/homes/ad.html?finnkode=205519251  205519251 2750000 map[Boligtype:Enebolig Bruksareal:160 m² Bruttoareal:181 m² Byggeår:1966 Eieform bolig:Eier (Selveier) Energimerking:
                        
                            G - oransje
                            
                     Etasje:3 Formuesverdi:744 510 kr Kommunale avg.:11 864 kr per år Mobil:
                      472 93 982
                         Omkostninger:70 092 kr Primærrom:122 m² Soverom:3 Søn.:31. januar Tomteareal:386 m² (eiet) Totalpris:2 820 092 kr  :12:00 - 13:00] false map[]}


713 <nil>